<a href="https://colab.research.google.com/github/drkoba/large-language-model-assisted-text-mining-LATeM-/blob/main/LATeM.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. Installation of Required Packages and Importing Libraries
# Installing necessary packages
!pip uninstall -y nltk
!pip uninstall -y pandas
!pip install --upgrade openai nltk pandas gensim pyLDAvis openpyxl xlsxwriter matplotlib networkx plotly scikit-learn PyPDF2
!pip install pandas

# Clearing the NLTK data directory
import nltk
import shutil
import os

nltk_data_dir = os.path.expanduser('~/nltk_data')
if os.path.exists(nltk_data_dir):
    shutil.rmtree(nltk_data_dir)

# Re-downloading NLTK data
nltk.download('punkt', force=True)
nltk.download('stopwords', force=True)

# Stopwords configuration (customize as needed)
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
# Add additional stopwords or words to remove
custom_stop_words = {
    'et', 'al', 'using', 'also', 'may', 'one', 'two', 'three',
    'could', 'would', 'should', 'due', 'eg', 'ie', 'table', 'figure',
    # Add patterns for numbers or years
    *map(str, range(1900, 2030)),  # Exclude years (from 1900 to 2029)
    '000', '00', '10', '20', '30', '40', '50', '60', '70', '80', '90',
    # Other unnecessary words
    'copyright', 'rights', 'reserved', 'http', 'doi', 'org', 'com', 'license', 'creativecommons', 'www','elsevier',"basel", "karger"
}
stop_words.update(custom_stop_words)

# Importing libraries
from nltk.tokenize import word_tokenize
import pandas as pd
import re
from collections import Counter, defaultdict
from gensim import corpora, models
import pyLDAvis.gensim_models
import pyLDAvis
from openai import OpenAI  # OpenAI client
import matplotlib.pyplot as plt
import networkx as nx
from google.colab import files
from sklearn.feature_extraction.text import CountVectorizer
from nltk.util import ngrams
import PyPDF2
import io

Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.3/472.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.4 MB/s eta 0:

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.11/dist-packages/PyPDF2/__init__.py:21: DeprecationWarning: PyPDF2 is deprecated. Please move to the pypdf library instead.
  warnings.warn(


In [ ]:
# @title 2. File Upload
# Step 1: File Upload
print("Please upload the Excel or CSV file you wish to analyze.")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Step 2: Data Loading
if file_name.endswith('.csv'):
    df = pd.read_csv(file_name)
elif file_name.endswith('.xlsx'):
    df = pd.read_excel(file_name)
else:
    raise ValueError("Unsupported file format. Please upload a CSV or Excel file.")

# Check for the presence of 'Abstract' and 'Publication Year' columns
if 'Abstract' not in df.columns or 'Publication Year' not in df.columns:
    raise ValueError("The file must include the 'Abstract' column and the 'Publication Year' column.")

分析したいExcelまたはCSVファイルをアップロードしてください。


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving scopus_1-4484_20250218_cleanup.csv to scopus_1-4484_20250218_cleanup.csv


In [ ]:
# @title 3. Text Preprocessing
# Step 3: Text Preprocessing and Tokenization
from nltk.tokenize import TreebankWordTokenizer

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove hyperlinks and email addresses
    text = re.sub(r'http\S+|www\S+|mailto:\S+', '', text)
    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenization
    tokenizer = TreebankWordTokenizer()
    tokens = tokenizer.tokenize(text)
    # Remove stopwords
    filtered_tokens = [word for word in tokens if word not in stop_words and len(word) > 1]
    return filtered_tokens

# Replace NaN values with an empty string
df['Abstract'] = df['Abstract'].fillna('')

# Add tokenized text as a new column
df['tokens'] = df['Abstract'].apply(preprocess_text)

# Creating n-grams (bigrams and trigrams)
def create_ngrams(tokens_list, n):
    ngrams_list = list(ngrams(tokens_list, n))
    ngrams_joined = ['_'.join(gram) for gram in ngrams_list]
    return ngrams_joined

# Add n-grams to the dataframe
df['bigrams'] = df['tokens'].apply(lambda x: create_ngrams(x, 2))
df['trigrams'] = df['tokens'].apply(lambda x: create_ngrams(x, 3))
df['all_tokens'] = df['tokens'] + df['bigrams'] + df['trigrams']

# Convert the 'Publication Year' column to integer type
df['Publication Year'] = pd.to_numeric(df['Publication Year'], errors='coerce')
df = df.dropna(subset=['Publication Year'])  # Drop rows where year is NaN
df['Publication Year'] = df['Publication Year'].astype(int)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Step 4: Specify the Time Periods for Analysis
periods = [ '2014-2016', '2017-2019', '2020-2022', '2023-2025']
period_ranges = {
    '2014-2016': (2014, 2016),
    '2017-2019': (2017, 2019),
    '2020-2022': (2020, 2022),
    '2023-2025': (2023, 2025)
}

# Add a 'period' column based on 'Publication Year'
def assign_period(year):
    for period, (start, end) in period_ranges.items():
        if start <= year <= end:
            return period
    return None  # If not applicable

df['period'] = df['Publication Year'].apply(assign_period)

# Drop rows where 'period' is None
df = df.dropna(subset=['period'])

# @title 4. Analysis of Frequent Words
# Step 5: Analyze frequent words and calculate new terms and terms with high growth rates

def analyze_terms(df, token_column):
    # Obtain the list of periods (sorted order)
    periods_sorted = sorted(df['period'].unique(), key=lambda x: period_ranges[x][0])

    # Create a dictionary to store tokenized data for each period
    period_tokens = {}
    for period in periods_sorted:
        period_df = df[df['period'] == period]
        tokens_list = period_df[token_column].tolist()
        period_tokens[period] = tokens_list

    # Calculate frequency for each period
    freq_by_period = {}
    for period in periods_sorted:
        all_tokens = [token for tokens in period_tokens[period] for token in tokens]
        freq_by_period[period] = Counter(all_tokens)

    # Store the top 50 frequent words for each period in a dataframe
    all_data = {}
    for period in periods_sorted:
        common_terms = freq_by_period[period].most_common(50)
        df_common = pd.DataFrame(common_terms, columns=['word', 'count'])
        all_data[period] = df_common

    # Calculate new terms for each period
    new_terms = {}
    previous_terms = set()
    for period in periods_sorted:
        current_terms = set(freq_by_period[period].keys())
        new_terms_in_period = current_terms - previous_terms
        new_terms[period] = new_terms_in_period
        previous_terms.update(current_terms)

    # Extract terms with high growth rates
    growth_terms = {}
    growth_rates = {}
    growth_counts = {}

    for i in range(1, len(periods_sorted)):
        current_period = periods_sorted[i]
        previous_period = periods_sorted[i-1]
        current_freq = freq_by_period[current_period]
        previous_freq = freq_by_period[previous_period]
        common_terms = set(current_freq.keys()) & set(previous_freq.keys())
        growth = {}
        counts = {}
        for term in common_terms:
            prev_count = previous_freq.get(term, 0)
            curr_count = current_freq.get(term, 0)
            if prev_count > 0:
                rate = ((curr_count - prev_count) / prev_count) * 100  # Convert to percentage
                growth[term] = rate
                counts[term] = curr_count  # Save frequency count
        # Sort in descending order of growth rate and extract top N
        top_n = 50  # Change as needed
        sorted_growth = sorted(growth.items(), key=lambda x: x[1], reverse=True)
        top_terms = sorted_growth[:top_n]
        growth_terms[current_period] = [term for term, rate in top_terms]
        growth_rates[current_period] = {term: rate for term, rate in top_terms}
        growth_counts[current_period] = {term: counts[term] for term, _ in top_terms}

    return all_data, new_terms, growth_terms, growth_rates, growth_counts, freq_by_period

# Unigram Analysis
print("\n=== Analysis of Unigrams (words) ===")
uni_all_data, uni_new_terms, uni_growth_terms, uni_growth_rates, uni_growth_counts, uni_freq_by_period = analyze_terms(df, 'tokens')

# Bigram Analysis
print("\n=== Analysis of Bigrams ===")
bi_all_data, bi_new_terms, bi_growth_terms, bi_growth_rates, bi_growth_counts, bi_freq_by_period = analyze_terms(df, 'bigrams')

# Trigram Analysis
print("\n=== Analysis of Trigrams ===")
tri_all_data, tri_new_terms, tri_growth_terms, tri_growth_rates, tri_growth_counts, tri_freq_by_period = analyze_terms(df, 'trigrams')

# Step 5.5: Displaying Results

# Display top 50 frequent unigrams for each period
print("\n=== Top 50 Frequent Unigrams for Each Period ===")
for period, df_common in uni_all_data.items():
    print(f"\nPeriod: {period}")
    print(df_common[['word', 'count']])

# Display top 50 frequent bigrams for each period
print("\n=== Top 50 Frequent Bigrams for Each Period ===")
for period, df_common in bi_all_data.items():
    print(f"\nPeriod: {period}")
    print(df_common[['word', 'count']])

# Display top 50 frequent trigrams for each period
print("\n=== Top 50 Frequent Trigrams for Each Period ===")
for period, df_common in tri_all_data.items():
    print(f"\nPeriod: {period}")
    print(df_common[['word', 'count']])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



=== ユニグラム（単語）の分析 ===

=== バイグラム（bigrams）の分析 ===

=== トライグラム（trigrams）の分析 ===

=== 各期間の頻出語トップ50（単語） ===

期間: 2014-2016
              word  count
0        aflatoxin    985
1          samples    881
2              afb    843
3             food    515
4       mycotoxins    457
5       aflatoxins    438
6        detection    383
7           levels    367
8    contamination    363
9            study    347
10            milk    338
11             gkg    318
12             afm    316
13          method    305
14         results    288
15        analysis    257
16            used    252
17       mycotoxin    235
18    respectively    234
19            feed    230
20            high    227
21           total    219
22           maize    218
23        exposure    218
24     aspergillus    217
25           found    205
26             ota    203
27        products    199
28          flavus    197
29       different    194
30          showed    193
31   concentration    192
32           limit    1

In [ ]:
# @title 5. New Terms
# Display new terms (unigrams)
print("\n=== Top 50 New Terms (Unigrams) ===")
for period in uni_new_terms:
    # New terms
    words = list(uni_new_terms[period])
    # Get frequency counts
    counts = [uni_freq_by_period[period][term] for term in words]
    df_new = pd.DataFrame({
        'word': words,
        'count': counts
    })
    df_new = df_new.sort_values(by='count', ascending=False).head(50)
    print(f"\nPeriod: {period}")
    print(df_new[['word', 'count']])

# Display new terms (bigrams)
print("\n=== Top 50 New Terms (Bigrams) ===")
for period in bi_new_terms:
    words = list(bi_new_terms[period])
    counts = [bi_freq_by_period[period][term] for term in words]
    df_new = pd.DataFrame({
        'word': words,
        'count': counts
    })
    df_new = df_new.sort_values(by='count', ascending=False).head(50)
    print(f"\nPeriod: {period}")
    print(df_new[['word', 'count']])

# Display new terms (trigrams)
print("\n=== Top 50 New Terms (Trigrams) ===")
for period in tri_new_terms:
    words = list(tri_new_terms[period])
    counts = [tri_freq_by_period[period][term] for term in words]
    df_new = pd.DataFrame({
        'word': words,
        'count': counts
    })
    df_new = df_new.sort_values(by='count', ascending=False).head(50)
    print(f"\nPeriod: {period}")
    print(df_new[['word', 'count']])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



=== 新規出現語トップ50（単語） ===

期間: 2014-2016
                word  count
922        aflatoxin    985
6602         samples    881
3071             afb    843
6306            food    515
4552      mycotoxins    457
3570      aflatoxins    438
1111       detection    383
7058          levels    367
7645   contamination    363
8690           study    347
2181            milk    338
2411             gkg    318
6900             afm    316
4768          method    305
719          results    288
6594        analysis    257
3204            used    252
2837       mycotoxin    235
5111    respectively    234
8450            feed    230
7431            high    227
2525           total    219
8277        exposure    218
1070           maize    218
303      aspergillus    217
4501           found    205
8092             ota    203
7485        products    199
8888          flavus    197
59         different    194
6399          showed    193
2693   concentration    192
4719           limit    188
8820     

In [ ]:
# @title 6. Terms with High Growth Rates
# Display high growth rate terms (unigrams)
print("\n=== Top 50 Terms with High Growth Rates (Unigrams) ===")
for period in uni_growth_terms:
    df_growth = pd.DataFrame({
        'word': uni_growth_terms[period],
        'growth_rate (%)': [uni_growth_rates[period][term] for term in uni_growth_terms[period]],
        'count': [uni_growth_counts[period][term] for term in uni_growth_terms[period]],
    })
    df_growth = df_growth.sort_values(by='growth_rate (%)', ascending=False)
    print(f"\nPeriod: {period}")
    print(df_growth[['word', 'growth_rate (%)', 'count']])

# Display high growth rate terms (bigrams)
print("\n=== Top 50 Terms with High Growth Rates (Bigrams) ===")
for period in bi_growth_terms:
    df_growth = pd.DataFrame({
        'word': bi_growth_terms[period],
        'growth_rate (%)': [bi_growth_rates[period][term] for term in bi_growth_terms[period]],
        'count': [bi_growth_counts[period][term] for term in bi_growth_terms[period]],
    })
    df_growth = df_growth.sort_values(by='growth_rate (%)', ascending=False)
    print(f"\nPeriod: {period}")
    print(df_growth[['word', 'growth_rate (%)', 'count']])

# Display high growth rate terms (trigrams)
print("\n=== Top 50 Terms with High Growth Rates (Trigrams) ===")
for period in tri_growth_terms:
    df_growth = pd.DataFrame({
        'word': tri_growth_terms[period],
        'growth_rate (%)': [tri_growth_rates[period][term] for term in tri_growth_terms[period]],
        'count': [tri_growth_counts[period][term] for term in tri_growth_terms[period]],
    })
    df_growth = df_growth.sort_values(by='growth_rate (%)', ascending=False)
    print(f"\nPeriod: {period}")
    print(df_growth[['word', 'growth_rate (%)', 'count']])


=== 増加率の高い語句トップ50（単語） ===

期間: 2017-2019
               word  growth_rate (%)  count
0         plantarum           3400.0     35
1              pigs           3000.0     31
2        intestinal           2700.0     56
3            larvae           2300.0     24
4               ros           2100.0     22
5              coli           2000.0     42
6               mip           1800.0     19
7              afls           1800.0     19
8               tht           1700.0     18
9           piglets           1650.0     35
10               zn           1600.0     17
11            tract           1600.0     17
12         severely           1600.0     17
13            swine           1600.0     17
14      escherichia           1600.0     17
15           serbia           1500.0     32
16         cellfree           1400.0     15
17          weather           1400.0     15
18          acetate           1400.0     15
19         external           1400.0     15
20  physicochemical           1400

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# @title 7. Topic Modeling (Including Unigrams and n-grams)
# Step 6: Perform Topic Modeling

# Setting the number of topics
num_topics = 5  # Change as needed

# Create dictionary and corpus (using all tokens)
texts = df['all_tokens'].tolist()
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Create LDA model
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10, random_state=42)

# Retrieve topics (increase the number of words per topic)
topics = lda_model.show_topics(num_topics=num_topics, num_words=15, formatted=False)

# Display topics
print("\n=== Topics ===")
for idx, topic in topics:
    topic_terms = ', '.join([term for term, _ in topic])
    print(f"Topic {idx+1}: {topic_terms}")

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



=== トピックの表示 ===
トピック 1: afb, detection, aflatoxin, samples, method, food, mycotoxins, aflatoxins, results, ota, study, aflatoxin_afb, analysis, used, developed
トピック 2: biochar, sn_kg, kg_ww, sn_kg_ww, mp_ingestion, csloeonp, ww, ibs, msat, oryzae_bl, afbngua, foqds, marbled_electric_ray, marbled_electric, electric_ray
トピック 3: afm, milk, samples, milk_samples, ngl, dairy, raw_milk, raw, aflatoxin_afm, pasteurized, iran, kg, cheese, afm_milk, mean
トピック 4: samples, gkg, mycotoxins, levels, feed, aflatoxin, aflatoxins, contamination, total, study, afs, ota, found, mycotoxin, food
トピック 5: aflatoxin, food, contamination, samples, mycotoxins, aflatoxins, flavus, study, aspergillus, production, fungal, health, maize, levels, results


In [ ]:
# Step 9: Comprehensive Analysis Using ChatGPT API

# Instantiate the OpenAI client (set API key)
client = OpenAI(api_key='')  # Enter your API key here

# Instantiate the OpenAI client (set API key)
#.api_key=''  # Enter your API key here

# @title OpenAI API Settings
# Setting the OpenAI API key
# For security reasons, avoid hardcoding the API key in the code.
# Use environment variables or another secure method.
# Here, as an example, prompt the user for input.

#import getpass

#openai_api_key = getpass.getpass("Please enter your OpenAI API key: ")
#openai.api_key = openai_api_key

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# @title 8. Comprehensive Analysis Using ChatGPT API
# Step 7: Comprehensive Analysis Using ChatGPT API
# Compile the analysis results
analysis_input = ""

# Append frequent word information (unigrams, bigrams, trigrams)
for period in periods:
    analysis_input += f"\nTop 50 Frequent Unigrams ({period}):\n"
    df_common_uni = uni_all_data[period]
    analysis_input += df_common_uni[['word', 'count']].to_string(index=False)

    analysis_input += f"\nTop 50 Frequent Bigrams ({period}):\n"
    df_common_bi = bi_all_data[period]
    analysis_input += df_common_bi[['word', 'count']].to_string(index=False)

    analysis_input += f"\nTop 50 Frequent Trigrams ({period}):\n"
    df_common_tri = tri_all_data[period]
    analysis_input += df_common_tri[['word', 'count']].to_string(index=False)

# Append new term information (unigrams, bigrams, trigrams)
for period in periods:
    # New terms (unigrams)
    words_uni = list(uni_new_terms.get(period, []))
    counts_uni = [uni_freq_by_period[period][term] for term in words_uni]
    df_new_uni = pd.DataFrame({'word': words_uni, 'count': counts_uni}).sort_values(by='count', ascending=False).head(50)

    # New terms (bigrams)
    words_bi = list(bi_new_terms.get(period, []))
    counts_bi = [bi_freq_by_period[period][term] for term in words_bi]
    df_new_bi = pd.DataFrame({'word': words_bi, 'count': counts_bi}).sort_values(by='count', ascending=False).head(50)

    # New terms (trigrams)
    words_tri = list(tri_new_terms.get(period, []))
    counts_tri = [tri_freq_by_period[period][term] for term in words_tri]
    df_new_tri = pd.DataFrame({'word': words_tri, 'count': counts_tri}).sort_values(by='count', ascending=False).head(50)

    analysis_input += f"\nNew Terms (Unigrams) ({period}):\n"
    analysis_input += df_new_uni.to_string(index=False)
    analysis_input += f"\nNew Terms (Bigrams) ({period}):\n"
    analysis_input += df_new_bi.to_string(index=False)
    analysis_input += f"\nNew Terms (Trigrams) ({period}):\n"
    analysis_input += df_new_tri.to_string(index=False)
    analysis_input += "\n"

# Append high growth rate term information (unigrams, bigrams, trigrams)
for period in periods:
    if period in uni_growth_terms:
        # High growth rate terms (unigrams)
        df_growth_uni = pd.DataFrame({
            'word': uni_growth_terms[period],
            'growth_rate (%)': [uni_growth_rates[period][term] for term in uni_growth_terms[period]],
            'count': [uni_growth_counts[period][term] for term in uni_growth_terms[period]],
        }).sort_values(by='growth_rate (%)', ascending=False).head(50)

        # High growth rate terms (bigrams)
        df_growth_bi = pd.DataFrame({
            'word': bi_growth_terms[period],
            'growth_rate (%)': [bi_growth_rates[period][term] for term in bi_growth_terms[period]],
            'count': [bi_growth_counts[period][term] for term in bi_growth_terms[period]],
        }).sort_values(by='growth_rate (%)', ascending=False).head(50)

        # High growth rate terms (trigrams)
        df_growth_tri = pd.DataFrame({
            'word': tri_growth_terms[period],
            'growth_rate (%)': [tri_growth_rates[period][term] for term in tri_growth_terms[period]],
            'count': [tri_growth_counts[period][term] for term in tri_growth_terms[period]],
        }).sort_values(by='growth_rate (%)', ascending=False).head(50)

        analysis_input += f"\nHigh Growth Rate Terms (Unigrams) ({period}):\n"
        analysis_input += df_growth_uni.to_string(index=False)
        analysis_input += f"\nHigh Growth Rate Terms (Bigrams) ({period}):\n"
        analysis_input += df_growth_bi.to_string(index=False)
        analysis_input += f"\nHigh Growth Rate Terms (Trigrams) ({period}):\n"
        analysis_input += df_growth_tri.to_string(index=False)
        analysis_input += "\n"

# Append topic information
analysis_input += "\nTopic Information:\n"
for idx, topic in topics:
    topic_terms = ', '.join([term for term, _ in topic])
    analysis_input += f"Topic {idx+1}: {topic_terms}\n"

# Create the prompt
prompt = f"""
You are an expert in natural language processing and research trend analysis. Based on the data analysis results provided below, please deeply consider the research trends.

### Data Analysis Results:

{analysis_input}

### Task:

1. **Analysis of Frequent Words**:
    - Analyze the top frequent words (unigrams, bigrams, trigrams) for each period and discuss the major research themes or trends indicated by these terms.
2. **Analysis of New Terms**:
    - For the top new terms (unigrams, bigrams, trigrams) for each period, consider the background of their appearance and the new research areas they suggest.
3. **Analysis of Terms with High Growth Rates**:
    - Analyze the terms with high growth rates (unigrams, bigrams, trigrams) for each period, discussing the factors behind their rapid increase and their impact on the research field.

4. **Identification of Major Research Topics**:
    - Integrate the frequent words, new terms, terms with high growth rates, and the results of the topic model to extract the major research topics, and explain their development and changes.

5. **Recent Trends and Future Outlook**:
    - Identify recent new trends and notable research areas, and discuss the potential impact on future research.

6. **Recommendations**:
    - Based on the analysis results, provide specific recommendations for future research or applications.

### Constraints:
- Do not include any items not mentioned in the analysis results.
- If the analysis contains publisher or author details, ignore them as they are words that should have been removed.

### Description of the Results:

- The analysis should be described concisely and logically, including specific figures or examples.
- For the recent trends, future outlook, and recommendations, summarize the key trends or points in a bullet list of 5 to 10 items.
"""
# Generate the analysis using the ChatGPT API
response = client.chat.completions.create(
    model="o3-mini-2025-01-31",
    messages=[
        {"role": "system", "content": "You are a data scientist with advanced analytical skills."},
        {"role": "user", "content": prompt}
    ],
    )

# Retrieve the generated analysis result
analysis_result = response.choices[0].message.content.strip()

# Display the analysis result
print("\n### Comprehensive Analysis Result ###\n")
print(analysis_result)

# Save the analysis result to a text file
with open('analysis_result.txt', 'w', encoding='utf-8') as f:
  f.write(analysis_result)
# Download the text file
files.download('analysis_result.txt')  # Confirm that 'files' has been imported from google.colab

# @title 9. Upload and Read Existing Review Papers
from google.colab import files

print("Please upload multiple PDF files of review papers for comparison.")
uploaded_reviews = files.upload()

import PyPDF2

def extract_text_from_pdf(file):
    reader = PyPDF2.PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

# Store the text of review papers in a list
review_texts = []
for file_name in uploaded_reviews.keys():
    with open(file_name, 'rb') as f:
        text = extract_text_from_pdf(f)
        review_texts.append(text)

# Create a dataframe for the text data
reviews_df = pd.DataFrame({'Review_Title': list(uploaded_reviews.keys()), 'Text': review_texts})


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



### 総合分析結果 ###

以下、各タスクに沿って分析結果から読み取れる示唆と考察を述べる。

──────────────────────────────
1. 頻出語の分析

・各期間とも「afb」「aflatoxin」「samples」「food」「detection」「mycotoxins／aflatoxins」などに高頻度が認められる。特に、2014–2016ではaflatoxin関連語が上位にランクインし、食品や飼料中のカビ毒検出に向けた方法論（chromatography、mass_spectrometryなど）への関心が示唆される。  
・2017–2019、2020–2022、2023–2025と時期が進むにつれ、「method」や「analysis」「results」といった言及が引き続き高く、基礎的な定量分析および検出技術の改善が主要テーマとなっている。  
・バイグラムやトライグラムでは「aflatoxin_afb」「liquid_chromatography」「highperformance_liquid_chromatography」など、実験手法や定量技術に焦点が当たっている。一方、後期になるほど「food_safety」「public_health」や「detection_limit」「ng_ml」など、実用的な安全基準や検出感度の向上を重視する傾向が強い。

──────────────────────────────
2. 新規出現語の分析

・2014–2016期では、基礎的な語句（aflatoxin, method, studyなど）のほか、実験的な検出手法（immunosorbent_assay, chromatography_hplcなど）が新規に出現し、従来手法の発展を示唆。  
・2017–2019期では、従来のaflatoxin関連ワードに加え、動物由来サンプル（donkey, piglets, intestinalなど）や食品加工、微生物群（cpa, cfs, eos, microbiomeなど）の言及が見受けられ、食品安全および動物健康への視点も拡大している。  
・2020–2022期では、一部で「machine_learning」や「aptamer_sensor」など先端技術の示唆に加え、nanoemu

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# @title 9. Upload and Read Existing Review Papers
from google.colab import files

print("Please upload multiple PDF files of review papers for comparison.")
uploaded_reviews = files.upload()

import PyPDF2

def extract_text_from_pdf(file):
    reader = PyPDF2.PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

# Store the text of review papers in a list
review_texts = []
for file_name in uploaded_reviews.keys():
    with open(file_name, 'rb') as f:
        text = extract_text_from_pdf(f)
        review_texts.append(text)

# Create a dataframe for the text data
reviews_df = pd.DataFrame({'Review_Title': list(uploaded_reviews.keys()), 'Text': review_texts})


比較対象となるレビュー論文のPDFファイルを複数アップロードしてください。


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving fmicb-11-01916.pdf to fmicb-11-01916.pdf
Saving foods-09-00518.pdf to foods-09-00518.pdf
Saving foods-12-00527.pdf to foods-12-00527.pdf


In [ ]:
# @title 10. Summarization and Key Points Extraction from Review Papers
import openai

# Setting the OpenAI API key (it is recommended to use environment variables or other secure methods)
#openai.api_key = 'YOUR_OPENAI_API_KEY'  # It is recommended to load from environment variables

def summarize_text(text, max_length=1500):
    response = client.chat.completions.create(
        model="o3-mini-2025-01-31",
        messages=[
            {"role": "system", "content": "You are an experienced researcher skilled at summarizing scientific papers."},
            {"role": "user", "content": f"Please concisely summarize the key points of the following review paper.\n\n{text}"}
        ],
    )
    summary = response.choices[0].message.content.strip()
    return summary

# Generate a summary for each review paper
reviews_df['Summary'] = reviews_df['Text'].apply(lambda x: summarize_text(x))

# Extraction of major trends or themes
def extract_key_points(summary):
    prompt = f"""
    From the following summary, extract the major research trends or themes in bullet points.

    ### Summary:
    {summary}

    ### Extracted Points:
    - List 3 to 5 major trends or themes in bullet points.
    """
    response = client.chat.completions.create(
        model="o3-mini-2025-01-31",
        messages=[
            {"role": "system", "content": "You are a researcher with advanced natural language processing skills."},
            {"role": "user", "content": prompt}
        ],
        )
    key_points = response.choices[0].message.content.strip()
    return key_points

reviews_df['Key_Points'] = reviews_df['Summary'].apply(lambda x: extract_key_points(x))

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# @title 11. Comparison of Data Analysis Results and Trends/Themes from Review Papers
# Create a summary of your analysis results
your_analysis_summary = """
### Summary of Your Data Analysis Results:
1. **Frequent Words Analysis**:
    - A list of top frequent words (unigrams, bigrams, trigrams) for each period.
2. **New Terms Analysis**:
    - A list of top new terms (unigrams, bigrams, trigrams) for each period.
3. **High Growth Rate Terms Analysis**:
    - A list of terms with high growth rates (unigrams, bigrams, trigrams) for each period.
4. **Major Research Topics**:
    - The major research topics extracted from the topic modeling results.
"""

# Combine key points from review papers
review_key_points = reviews_df['Key_Points'].tolist()
review_combined_key_points = "\n".join([f"### Major Trends or Themes for Review Paper {i+1}:\n{kp}" for i, kp in enumerate(review_key_points, 1)])

# Create the comparison prompt
comparison_prompt = f"""
You are an expert in evaluating the validity of research methodologies. Based on the following data analysis results and the major trends or themes extracted from existing review papers, please compare them and evaluate the validity of your analysis method.

### Your Data Analysis Results:
{your_analysis_summary}

### Major Trends or Themes from Review Papers:
{review_combined_key_points}

### Task:

1. **Evaluation of Consistency**:
    - Assess the degree of agreement between your data analysis results and the trends/themes from the review papers.
    - Specify the points of agreement and disagreement.

2. **Validity of the Method**:
    - Based on the points of agreement and disagreement, assess how valid your analysis method is.

3. **Suggestions for Improvement**:
    - Discuss the strengths of your analysis method and the areas that need improvement.

### Constraints:
- Do not include any information not mentioned in the review papers.
- Use specific figures or examples from the analysis results and review papers in your evaluation.

### Description of the Results:
- Present the evaluation in a concise and logical manner, clearly addressing each evaluation point.
"""

# Generate the evaluation using ChatGPT
response_comparison = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an experienced expert in evaluating research methodologies."},
        {"role": "user", "content": comparison_prompt}
    ],
    temperature=0.7,
    max_tokens=2000,
)

comparison_result = response_comparison.choices[0].message.content.strip()

# Display the evaluation result
print("\n### Validity Evaluation Report ###\n")
print(comparison_result)

# Save the evaluation report to a text file
with open('validation_report.txt', 'w', encoding='utf-8') as f:
    f.write(comparison_result)

# Download the text file
files.download('validation_report.txt')

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



### 妥当性評価レポート ###

### 1. 整合性の評価：

#### 一致点：
- **サンプル前処理技術**：
  - あなたのデータ分析結果の「新規出現語の分析」や「増加率の高い語句の分析」で「QuEChERS」や「液液抽出」、「固相抽出」といった前処理技術が頻繁に出現し、レビュー論文2、3、4でもこれらの技術の重要性が強調されています。
  
- **分離・検出技術の進展**：
  - トピックモデリングで抽出された「クロマトグラフィー」や「MS/MS」などの技術は、レビュー論文全体で高感度な分析法として繰り返し言及されています。特にLC-MS/MSに関する言及が多く、一貫したトレンドが見られます。

- **免疫学的手法とバイオセンサー技術**：
  - データ分析結果には「ELISA」や「バイオセンサー」に関連する語句が含まれ、レビュー論文2、3でもこれらの技術が現場での迅速なスクリーニングに利用されているとされています。

#### 一致していない点：
- **新素材や先端技術**：
  - レビュー論文3には「新素材を利用した固相抽出」や「プロテオミクス・ゲノミクス」といった先端技術の活用について言及されていますが、あなたのデータ分析ではこれらの要素が明確に抽出されていないようです。

- **自動化・ミニチュア化**：
  - レビュー論文2で将来のトレンドとして「自動化・ミニチュア化」が挙げられていますが、あなたのデータ分析結果にはこの点に特化した語句が見当たりません。

### 2. 手法の妥当性：

あなたの分析手法は、主要な研究トピックの抽出において一定の妥当性を示しています。特に、頻出語やトピックモデリングによる主要テーマは、レビュー論文での傾向と一致しており、データの分析精度が高いと評価できます。しかし、全体的な技術の進展や将来のトレンドを捉えるにはやや弱い部分も見受けられます。特に、新素材の利用や自動化の流れなど、細分化されたトピックの抽出が不足しています。

### 3. 改善点の提案：

#### 強み：
- 頻出語やトピックモデリングを用いた分析により、レビュー論文で言及される主要な技術や手法を正確に捉えています。
- 新規出現語や増加率の高い語句の分析によって、技術の進化や新たなトレンドを一定程度把握できています。


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# @title 12. Detailed Validity Evaluation Report (Optional)
# Since a report was generated in Step 3, this section is for further customization if needed.
# For example, create a prompt to further detail the evaluation of the method.

additional_prompt = f"""
Below is the comparison result between your data analysis results and the major trends or themes extracted from existing review papers. Based on this information, please provide a more detailed evaluation of the validity of your method.

### Comparison Result:
{comparison_result}

### Task:

1. **Detailed Validity Evaluation**:
    - For each point of agreement and disagreement, provide detailed reasons and background.

2. **Method Improvement Suggestions**:
    - Considering the identified discrepancies, propose improvements or new approaches for the analysis method.

3. **Overall Conclusion**:
    - Summarize the overall validity of your analysis method and provide your outlook for future research.

### Constraints:
- Provide specific examples based on the review papers and data analysis results.
- Ensure the evaluation is objective and data-driven.

### Description of the Results:
- Present your evaluation in a concise and logical manner, clearly addressing each evaluation point.
"""

# Generate a detailed evaluation using ChatGPT
response_additional = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an experienced expert in evaluating research methodologies."},
        {"role": "user", "content": additional_prompt}
    ],
    temperature=0.7,
    max_tokens=2000,
)

additional_comparison_result = response_additional.choices[0].message.content.strip()

# Display the detailed evaluation result
print("\n### Detailed Validity Evaluation Report ###\n")
print(additional_comparison_result)

# Save the detailed evaluation report to a text file
with open('detailed_validation_report.txt', 'w', encoding='utf-8') as f:
    f.write(additional_comparison_result)

# Download the text file
files.download('detailed_validation_report.txt')

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



### 詳細な妥当性評価レポート ###

### 詳細な妥当性評価

#### 一致点の評価：

1. **サンプル前処理技術**：
   - **理由と背景**：QuEChERSや液液抽出、固相抽出は、分析化学における標準的なサンプル前処理技術です。これらの技術は多くの研究分野で広く用いられ、分析の精度と効率を向上させるために不可欠です。レビュー論文でもこれらの技術が繰り返し強調されているのは、これらが確立された手法であることを示しています。
   - **評価**：あなたのデータ分析結果がこれらの技術を頻繁に取り上げていることは、分析手法が主要な研究傾向を正確に反映していることを示しており、その妥当性が確認できます。

2. **分離・検出技術の進展**：
   - **理由と背景**：クロマトグラフィーやMS/MS（特にLC-MS/MS）は高感度で高精度な分析が可能なため、多くの研究で採用されています。これらの技術は、複雑なサンプルマトリックスからの成分分離と定量において非常に有効です。
   - **評価**：データ分析結果がこれらの技術を強調している点は、現代の分析技術の進展を正確に捉えていることを示し、レビュー論文の内容と整合性が高いです。

3. **免疫学的手法とバイオセンサー技術**：
   - **理由と背景**：ELISAやバイオセンサーは、迅速かつ簡便な分析が可能で、多くのフィールドにおけるスクリーニングに活用されています。特に、現場での即時分析の必要性が高まる中で、その利用が増加しているのは自然な流れです。
   - **評価**：分析結果にこれらの技術が含まれていることは、技術の実用性と現場でのニーズに対応していることを示し、妥当性が支持されます。

#### 一致していない点の評価：

1. **新素材や先端技術**：
   - **理由と背景**：レビュー論文3で言及される新素材やプロテオミクス・ゲノミクスは、比較的新しい研究領域であり、特定の専門分野に限定される可能性があります。これにより、一般的なデータセットではこれらのトピックが明確に抽出されにくい状況が考えられます。
   - **評価**：あなたのデータ分析でこれらが抽出されていないことは、データセットの特異性や分析手法の焦点が一般的な技術に偏っている可能性を示唆していま

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# @title 13. Save the Results
# Step 10: Save the results (export to an Excel file)
try:
    with pd.ExcelWriter('all_terms_analysis_results.xlsx', engine='xlsxwriter') as writer:
        # Save frequent unigrams to a sheet
        for period, df_common in uni_all_data.items():
            sheet_name = f'Uni_Frequent_{period}'
            if len(sheet_name) > 31:
                sheet_name = f'Uni_Freq_{period[:4]}-{period[-4:]}'
            df_common.to_excel(writer, sheet_name=sheet_name, index=False)

        # Save frequent bigrams to a sheet
        for period, df_common in bi_all_data.items():
            sheet_name = f'Bi_Frequent_{period}'
            if len(sheet_name) > 31:
                sheet_name = f'Bi_Freq_{period[:4]}-{period[-4:]}'
            df_common.to_excel(writer, sheet_name=sheet_name, index=False)

        # Save frequent trigrams to a sheet
        for period, df_common in tri_all_data.items():
            sheet_name = f'Tri_Frequent_{period}'
            if len(sheet_name) > 31:
                sheet_name = f'Tri_Freq_{period[:4]}-{period[-4:]}'
            df_common.to_excel(writer, sheet_name=sheet_name, index=False)

        # Save new terms (unigrams) to a sheet
        for period, new_terms in uni_new_terms.items():
            words = list(new_terms)
            counts = [uni_freq_by_period[period][term] for term in words]
            df_new = pd.DataFrame({
                'word': words,
                'count': counts
            }).sort_values(by='count', ascending=False).head(50)
            sheet_name = f'Uni_New_{period}'
            if len(sheet_name) > 31:
                sheet_name = f'Uni_New_{period[:4]}-{period[-4:]}'
            df_new.to_excel(writer, sheet_name=sheet_name, index=False)

        # Save new terms (bigrams) to a sheet
        for period, new_terms in bi_new_terms.items():
            words = list(new_terms)
            counts = [bi_freq_by_period[period][term] for term in words]
            df_new = pd.DataFrame({
                'word': words,
                'count': counts
            }).sort_values(by='count', ascending=False).head(50)
            sheet_name = f'Bi_New_{period}'
            if len(sheet_name) > 31:
                sheet_name = f'Bi_New_{period[:4]}-{period[-4:]}'
            df_new.to_excel(writer, sheet_name=sheet_name, index=False)

        # Save new terms (trigrams) to a sheet
        for period, new_terms in tri_new_terms.items():
            words = list(new_terms)
            counts = [tri_freq_by_period[period][term] for term in words]
            df_new = pd.DataFrame({
                'word': words,
                'count': counts
            }).sort_values(by='count', ascending=False).head(50)
            sheet_name = f'Tri_New_{period}'
            if len(sheet_name) > 31:
                sheet_name = f'Tri_New_{period[:4]}-{period[-4:]}'
            df_new.to_excel(writer, sheet_name=sheet_name, index=False)

        # Save high growth rate terms (unigrams) to a sheet
        for period, terms in uni_growth_terms.items():
            df_growth = pd.DataFrame({
                'word': terms,
                'growth_rate (%)': [uni_growth_rates[period][term] for term in terms],
                'count': [uni_growth_counts[period][term] for term in terms],
            }).sort_values(by='growth_rate (%)', ascending=False).head(50)
            sheet_name = f'Uni_Growth_{period}'
            if len(sheet_name) > 31:
                sheet_name = f'Uni_Growth_{period[:4]}-{period[-4:]}'
            df_growth.to_excel(writer, sheet_name=sheet_name, index=False)

        # Save high growth rate terms (bigrams) to a sheet
        for period, terms in bi_growth_terms.items():
            df_growth = pd.DataFrame({
                'word': terms,
                'growth_rate (%)': [bi_growth_rates[period][term] for term in terms],
                'count': [bi_growth_counts[period][term] for term in terms],
            }).sort_values(by='growth_rate (%)', ascending=False).head(50)
            sheet_name = f'Bi_Growth_{period}'
            if len(sheet_name) > 31:
                sheet_name = f'Bi_Growth_{period[:4]}-{period[-4:]}'
            df_growth.to_excel(writer, sheet_name=sheet_name, index=False)

        # Save high growth rate terms (trigrams) to a sheet
        for period, terms in tri_growth_terms.items():
            df_growth = pd.DataFrame({
                'word': terms,
                'growth_rate (%)': [tri_growth_rates[period][term] for term in terms],
                'count': [tri_growth_counts[period][term] for term in terms],
            }).sort_values(by='growth_rate (%)', ascending=False).head(50)
            sheet_name = f'Tri_Growth_{period}'
            if len(sheet_name) > 31:
                sheet_name = f'Tri_Growth_{period[:4]}-{period[-4:]}'
            df_growth.to_excel(writer, sheet_name=sheet_name, index=False)

        # Save topic information to a sheet
        topic_list = []
        for idx, topic in topics:
            topic_terms = ', '.join([term for term, _ in topic])
            topic_list.append({'Topic': f'Topic {idx+1}', 'Terms': topic_terms})
        df_topics = pd.DataFrame(topic_list)
        df_topics.to_excel(writer, sheet_name='Topics', index=False)

        # Save the generated analysis result to a sheet
        df_analysis_result = pd.DataFrame({'Analysis Result': [analysis_result]})
        df_analysis_result.to_excel(writer, sheet_name='Analysis_Result', index=False)

        # Save the validation result to a sheet
        ##df_validation_result = pd.DataFrame({'Validation Result': [validation_result]})
        ##df_validation_result.to_excel(writer, sheet_name='Validation_Result', index=False)

    # Download the file (if using Google Colab)
    files.download('all_terms_analysis_results.xlsx')
except Exception as e:
    print(f"An error occurred: {e}")

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>